In [26]:
from __future__ import division

#import affine
#import contextily
import matplotlib.colors as mcolors
import matplotlib.cm as cm

import fiona
import geopandas as gpd
import math

import matplotlib as mpl
from   matplotlib.colors import LinearSegmentedColormap
import matplotlib.cm as cm
import matplotlib.pyplot as plt
from   matplotlib.patches import Patch
from   matplotlib.ticker import FormatStrFormatter
import os
import numpy as np
import pyproj
import pandas as pd

import rasterio 
from   rasterio.crs import CRS
from   rasterio.enums import Resampling
from   rasterio.features import rasterize
import rasterio.mask
from   rasterio.plot import show
from   rasterio.transform import from_bounds
from   rasterio import shutil as rio_shutil
from   rasterio.vrt import WarpedVRT
from   rasterio import Affine

from   requests import Request
import seaborn as sns
from   scipy.interpolate import interpn
from   scipy.interpolate import griddata
from   scipy import stats
import sys
import shapely
from   shapely.geometry import Point
from   shapely.geometry import mapping, Polygon
import string
from   owslib.wcs import WebCoverageService



%matplotlib inline

In [27]:
filelocation = r'C:\Users\mar886\WaterTableProject\Willunga\hand_output_rasters\results_arrays'
figure_directory = r'C:\Users\mar886\WaterTableProject\Willunga\willunga_wt\figures\paper'

In [28]:
names_list = []
arrays_list = []

for file in os.listdir(filelocation):
    if file.endswith(".out"):
        names_list.append(file)
        ar = np.loadtxt(os.path.join(filelocation, file))
        arrays_list.append(ar)
    else:
        pass
    
print(names_list)
print(len(names_list))

['001.out', '002.out', '003.out', '004.out', '005.out', '008.out', '009.out', '012.out', '012b.out', '014.out', '016.out', '017.out', '021.out', '024.out', '025.out', '026.out', '028.out', '029.out', '030.out', '031.out', '032.out', '033.out', '034.out', '036.out', '037.out', '043.out', '044.out', '045.out', '047.out', '048.out', '049.out', '050.out', 'AAA3.out', 'AGC_001.out', 'AGC_011.out', 'AGC_014.out', 'AGC_015.out', 'AGC_016.out', 'AGC_018.out', 'BARD.out', 'BB01.out', 'C01.out', 'DEW_01.out', 'DEW_02.out', 'DEW_03.out', 'DEW_04.out', 'EMM01.out', 'E_0123.out', 'GH01.out', 'J5897.out', 'JAR.out', 'LW.out', 'NB.out', 'R01.out', 'R051.out', 'RS01.out', 'SJ01.out', 'TV_001.out', 'UWA_001.out', 'UWA_002.out', 'UWA_003.out', 'U_1949.out', 'XZ01.out']
63


# Calulate statistics

In [29]:
wt_array_stack = np.dstack(arrays_list)
wt_array_stack = np.moveaxis(wt_array_stack, source=2, destination=0)

In [30]:
max_array = np.amax(wt_array_stack, axis=0)
min_array = np.amin(wt_array_stack, axis=0)
range_array = max_array - min_array
std_array = np.nanstd(wt_array_stack, axis=0)
mean_array = np.nanmean(wt_array_stack, axis=0)

C:\Users\mar886\Anaconda3\envs\geo_env\lib\site-packages\numpy\lib\nanfunctions.py:1664: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
C:\Users\mar886\AppData\Local\Temp/ipykernel_18828/1839270943.py:5: RuntimeWarning: Mean of empty slice
  mean_array = np.nanmean(wt_array_stack, axis=0)


# Pull in well indices

In [31]:
f_well_ixs = os.path.join(r'C:\Users\mar886\WaterTableProject\Willunga\willunga_wt\output_data',
                         "df_well_ixs.csv")

df_well_ixs = pd.read_csv(f_well_ixs)

df_well_ixs = df_well_ixs.drop(columns=['DHNO.1'])
df_well_ixs.head()

,DHNO,rswl,well_ix,well_ix_x,well_ix_y,letter
0,26558,4.09,"(186, 20)",186,20,NaN
1,26565,4.33,"(155, 40)",155,40,NaN
2,26585,18.22,"(144, 56)",144,56,NaN
3,27050,5.54,"(195, 50)",195,50,NaN
4,27063,15.90,"(217, 35)",217,35,NaN


# Calibration data

In [32]:
# Data frame of values without letters
df_well_ixs_c = df_well_ixs[df_well_ixs['letter'].isna()]
df_well_ixs_c = df_well_ixs_c.copy()
df_well_ixs_c.index = df_well_ixs_c.DHNO

In [33]:
calibrated_h_list = []
ar_name_list = []

for ar_i in range(len(arrays_list)):
        
    wt_ar = arrays_list[ar_i]
    nm = names_list[ar_i][:-4]
    well_hds = []

    plt.figure(figsize=(30,30))
    plt.imshow(wt_ar, cmap='rainbow')
    plt.colorbar()

    for ix in range(len(df_well_ixs_c)):
        xi = df_well_ixs_c.iloc[ix,3]
        yi = df_well_ixs_c.iloc[ix,4]
        h = wt_ar[xi, yi]
        plt.plot(yi,xi,"kx", markersize=10)
        plt.text(yi,xi,"%2.1f"%h, fontsize=15)

        well_hds.append(h)

    #print(well_hds)
    #plt.savefig(os.path.join(r'C:\Users\mar886\WaterTableProject\Willunga\willunga_wt\figures\rasters_with_wellvalues',
     #                       nm), dpi=300)
    plt.close()
    calibrated_h_list.append(well_hds)
    df_well_ixs_c[nm] = well_hds
    
    ar_name_list.append(nm)

In [34]:
df_well_ixs_c.columns

df_well_ixs_c['n'] = df_well_ixs_c[ar_name_list].count(axis=1)
df_well_ixs_c['mean'] = df_well_ixs_c[ar_name_list].mean(axis=1)


df_well_ixs_c['25th'] = df_well_ixs_c[ar_name_list].quantile(q=0.25,axis=1)
df_well_ixs_c['75th'] = df_well_ixs_c[ar_name_list].quantile(q=0.75,axis=1)

datanm = os.path.join(r'C:\Users\mar886\WaterTableProject\Willunga\willunga_wt\output_data',
                      "df_well_ixs_c.csv")
                      
df_well_ixs_c.to_csv(datanm)

# Calculate residuals

In [35]:
df_residuals = pd.DataFrame()
df_sq_residuals = pd.DataFrame()

for ar_i in range(len(ar_name_list)):
    
    nm = ar_name_list[ar_i]
    
    #print(nm)
    
    resid_column = []
    sq_resid_column = []
    
    for well_i in range(len(df_well_ixs_c)):
        rswl = df_well_ixs_c.loc[df_well_ixs_c.index[well_i], 'rswl']
        h_est = df_well_ixs_c.loc[df_well_ixs_c.index[well_i], nm]
        resid_column.append(h_est - rswl)
        sq_resid_column.append((h_est - rswl)**2)
        
    df_residuals[nm] = resid_column
    df_sq_residuals[nm] = sq_resid_column

In [36]:
df_residuals.index = df_well_ixs_c.index
df_sq_residuals.index = df_well_ixs_c.index
df_residuals['mean'] = df_residuals[ar_name_list].mean(axis=1)


In [37]:
df_residuals.head()

,001,002,003,004,005,008,009,012,012b,014,...,R051,RS01,SJ01,TV_001,UWA_001,UWA_002,UWA_003,U_1949,XZ01,mean
DHNO,,,,,,,,,,,,,,,,,,,,,
26558,-24.293262,-22.147812,-11.653098,-1.134928,-13.727142,-5.352213,4.905111,-3.535181,NaN,-20.796440,...,-1.958449,NaN,0.525174,-16.949505,-8.843154,0.720652,-5.123586,NaN,-1.174240,-5.247317
26565,-2.587511,-3.364882,0.572679,0.917573,-4.831065,1.637878,4.542720,2.357495,NaN,-2.490521,...,1.559466,2.251695,0.314110,1.873080,2.907988,2.529198,1.537774,3.233270,1.554896,1.725026
26585,-3.908103,-2.900313,-1.154391,-4.469781,-3.127524,-7.351429,-7.892585,-1.727881,-2.471982,-3.526679,...,-7.544464,-4.563859,-4.151810,-3.414855,-1.918898,-3.720938,-0.338855,-5.275029,-4.016737,-4.497387
27050,0.998256,-2.917102,3.561677,3.003658,3.060751,-0.518589,4.004954,0.948711,3.879268,3.138637,...,3.224072,2.702831,3.374801,1.504753,-0.892612,5.290378,2.511178,3.401176,3.504769,2.620018
27063,-5.383580,-3.115051,-4.220353,-3.425550,-2.113707,-2.055099,-3.967429,-14.321672,NaN,-4.517824,...,-5.108832,-0.036927,-4.587687,-4.502165,-9.961402,-2.053534,-4.239123,-5.657654,-5.873496,-3.602909


# Pull in "original" well data and add residuals, so I can plot in ArcGIS

In [41]:
f_well_latlong = os.path.join(r'C:\Users\mar886\WaterTableProject\Willunga\willunga_wt\output_data',
                         "well_summary_gdf_ss_c.csv")

df_well_latlong = pd.read_csv(f_well_latlong)
df_well_latlong.index = df_well_latlong.DHNO

In [45]:
df_well_latlong["mean_residual"] = np.nan

for ix in df_residuals.index:
    df_well_latlong.loc[ix, "mean_residual"] = "%2.1f"%df_residuals.loc[ix, "mean"] 

In [46]:
df_well_latlong.head()

,Unnamed: 0,DHNO,Unit_No,Unit_Number,dh_name,network,Obs_No,drillhole_class,water_point_type_code,Aquifer,...,salinity,water_chemistry,geophys_log,drill_log,lith_log,geometry,Datetime,dropped,new_well_locations,mean_residual
DHNO,,,,,,,,,,,,,,,,,,,,,
26558,26558,26558,6527-562,652700562.0,WLG 40,MCL_VALE,WLG040,WW,NaN,Te2,...,Y,Y,Y,Y,Y,POINT (268941.81 6093083.52),2021-08-09,N,N,-5.2
26565,26565,26565,6527-569,652700569.0,NaN,MCL_VALE,WLG044,WW,NaN,Te2,...,Y,Y,Y,Y,Y,POINT (270906.82 6096136.56),2021-10-09,N,N,1.7
26585,26585,26585,6527-589,652700589.0,NaN,MCL_VALE,WLG019,WW,NaN,Qpa,...,Y,N,Y,N,N,POINT (272506.29 6097255.78),2021-10-09,N,N,-4.5
27050,27050,27050,6527-1054,652701054.0,MAR SITE 12,MCL_VALE,WLG049,WW,NaN,Tomw,...,Y,Y,N,Y,N,POINT (271897.13 6092138.11),2021-11-19,N,N,2.6
27063,27063,27063,6527-1067,652701067.0,NaN,MCL_VALE,WLG057,WW,NaN,Qpa,...,Y,Y,Y,Y,Y,POINT (270438.32 6089954.64),2021-10-09,N,N,-3.6


In [47]:
# Save as csv so I can pull into arcgis

df_well_latlong.to_csv(os.path.join(r'C:\Users\mar886\WaterTableProject\Willunga\willunga_wt\output_data\residuals_data',
                                   "df_residuals.csv"))

# Calculating statistics

In [ ]:
# Residuals - - -
df_residuals['n'] = df_residuals[ar_name_list].count(axis=1)
df_residuals['mean'] = df_residuals[ar_name_list].mean(axis=1)
df_residuals['median'] = df_residuals[ar_name_list].median(axis=1)
df_residuals['sq_mean'] = df_residuals[ar_name_list].mean(axis=1)**2
df_residuals['norm_mean'] = df_residuals[ar_name_list].mean(axis=1)/df_well_ixs_c['rswl']
df_residuals['sq_norm_mean'] = df_residuals[ar_name_list].mean(axis=1)**2/df_well_ixs_c['rswl']**2
df_residuals['rt_sq_mean'] = np.sqrt(df_residuals[ar_name_list].mean(axis=1)**2)


df_residuals['25th'] = df_residuals[ar_name_list].quantile(q=0.25,axis=1)
df_residuals['75th'] = df_residuals[ar_name_list].quantile(q=0.75,axis=1)